## Overview

Banks, merchants and credit card processors companies lose billions of dollars every year to credit card fraud. Credit Card Fraud happens through a variety of methods involving:

Lost or stolen cards used for online transactions;
Rogue Merchants that place fraudulent charges on customer purchases;
Manual Skimming of ATMs which can also be done by criminal merchants with POS machines. Credit card information of customers are stolen by skimmers and used for illegitimate transactions.

Graph databases have proven to be really powerful in detecting credit card fraud faster by representing transactions as a graph, thus giving quick insights to common denominators and helping identify the point of origin of the scam.




## Methodology
 
Working with a synthetic credit card transaction dataset while leveraging Neo4J’s Graph Data Science capabilities, our approach is to query the dataset to find common denominators for the fraudulent transactions. Since the dataset involves purchases made using credit cards, there are patterns to watch out for or red flags that raises suspicions in transactions;

Deviation from customers usual spending habit;
Transactions within a particular time window;
Merchant serving as denominator for repeated fraud transactions within a time window.
Identify point of origin of fraud involving credit card and merchants involved in all fraud activities.


## Data Manipulation and Preprocessing

A series of credit card transactions can be represented as a graph. Each transaction involves two nodes: a person (the customer) and a merchant. The nodes are linked by the transaction itself (which is a node) but also a connection between the customer and merchant.

To create relational graph from dataset, we will load the dataset and engineer it to derive relevant nodes and relationships.


## Load Dataset

The synthetic credit card transaction dataset has the following attributes;

1 index ⇒ Unique identifier to identify each row or each transaction;
2 trans_date ⇒ Transaction Date;
3 trans_time ⇒ Transaction Time;
4 cc_num ⇒ Credit Card Number of Customer;
5 merchant ⇒  Merchant Name;
6 category ⇒ Category of Merchant;
7 amt ⇒ Amount of Transaction;
8 first ⇒  First Name of Credit Card Holder;
9 last ⇒ Last Name of Credit Card Holder;
10 gender ⇒  Gender of Credit Card Holder;
11 street ⇒ Street Address of Credit Card Holder;
12 city ⇒ City of Credit Card Holder;
13 state ⇒  State of Credit Card Holder;
14 zip ⇒ Zip code of Credit Card Holder;
15 lat ⇒ Latitude Location of Credit Card Holder;
16 long ⇒ Longitude Location of Credit Card Holder;
17 city_pop ⇒ Credit Card Holder's City Population;
18 job ⇒ Job of Credit Card Holder;
19 dob ⇒  Date of Birth of Credit Card Holder;
20 trans_num ⇒  Transaction Number;
21 unix_time ⇒  UNIX Time of transaction;
22 merch_lat ⇒  Latitude Location of Merchant;
23 merch_long ⇒ Longitude Location of Merchant;
24 is_fraud ⇒ Fraud Flag;


Transform dataset using Pandas to create subsets or tables which correspond to nodes to be loaded to neo4j. Our dataset will be feature engineered to create customer, merchant and purchases (transactions) records.

In order to engineer our dataset and to execute Cypher queries, make sure that the library `pandas`, `py2neo` and the IPython extension `icypher` are installed.
If not, run the following command to install them:


In [1]:
pip install pandas icypher py2neo


Note: you may need to restart the kernel to use updated packages.


## Import pandas library to load and engineer the dataset to create the relevant subsets 

In [1]:
import pandas as pd
fraudDF = pd.read_csv("./fraudTrain.csv") # load dataset as dataframe
fraudDF.head(5)     # display first 5 records in dataset 

,Unnamed: 0,trans_date,trans_time,cc_num,merchant,category,amt,first,last,gender,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,01/01/19,00:00:18,2703186189652090,"Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,01/01/19,00:00:44,630423337322,"Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,01/01/19,00:00:51,38859492057661,Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,01/01/19,00:01:16,3534093764340240,"Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,01/01/19,00:03:06,375534208663984,Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


## Create customers table with unique credit card belong to customer and drop attributes not relevant to analysis

In [3]:
sampleData = fraudDF.copy() # create a copy of the data to work with

unique_customers = sampleData.drop_duplicates(subset="cc_num")  # reduce data to entries with unique cc_num
unique_customers["name"] = unique_customers["first"] + " " + unique_customers["last"]     # merge first and last names to create fullname
unique_customers.drop(columns=["trans_date", "trans_time", "merchant", "category", "amt",
            "lat", "long", "city_pop", "street", "city", "dob", "job", "is_fraud", "first", "last",
                             "Unnamed: 0",  "trans_num", "unix_time", "merch_lat", "merch_long", "zip"                   ],
                       inplace=True)      # drop irrelevant attributes


unique_customers.head(5)    #view first 5 records of unique customers

/tmp/ipykernel_47116/1928869086.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_customers["name"] = unique_customers["first"] + " " + unique_customers["last"]     # merge first and last names to create fullname
/tmp/ipykernel_47116/1928869086.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_customers.drop(columns=["trans_date", "trans_time", "merchant", "category", "amt",


,cc_num,gender,state,name
0,2703186189652090,F,NC,Jennifer Banks
1,630423337322,F,WA,Stephanie Gill
2,38859492057661,M,ID,Edward Sanchez
3,3534093764340240,M,MT,Jeremy White
4,375534208663984,M,VA,Tyler Garcia


## Create merchants table with unique merchant and drop attributes not relevant to analysis

In [4]:
unique_merchants = sampleData.drop_duplicates(subset="merchant")  # reduce data to entries with unique merchants
unique_merchants.drop(columns=["trans_date", "trans_time", "amt", "cc_num", "gender", "state",
            "lat", "long", "city_pop", "street", "city", "dob", "job", "is_fraud", "first", "last",
                             "Unnamed: 0",  "trans_num", "unix_time", "merch_lat", "merch_long", "zip"                   ],
                       inplace=True)      # drop irrelevant attributes


unique_merchants.head(5)    #view first 5 records of unique customers

/tmp/ipykernel_47116/1355002207.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_merchants.drop(columns=["trans_date", "trans_time", "amt", "cc_num", "gender", "state",


,merchant,category
0,"Rippin, Kub and Mann",misc_net
1,"Heller, Gutmann and Zieme",grocery_pos
2,Lind-Buckridge,entertainment
3,"Kutch, Hermiston and Farrell",gas_transport
4,Keeling-Crist,misc_pos


## Create purchases table with unique transaction number and drop attributes not relevant to analysis


In [5]:
# sampleData["unix_time"].unique()[1].dtype

In [5]:
unique_purchases = sampleData.drop_duplicates(subset="trans_num")  # reduce data to entries with unique purchases
unique_purchases.drop(columns=["gender", "state", "category",
            "lat", "long", "city_pop", "street", "city", "dob", "job", "first", "last",
                             "Unnamed: 0", "merch_lat", "merch_long", "zip"                   ],
                       inplace=True)      # drop irrelevant attributes


unique_purchases.head(5)    #view first 5 records of unique purchases

,trans_date,trans_time,cc_num,merchant,amt,trans_num,unix_time,is_fraud
0,01/01/19,00:00:18,2703186189652090,"Rippin, Kub and Mann",4.97,0b242abb623afc578575680df30655b9,1325376018,0
1,01/01/19,00:00:44,630423337322,"Heller, Gutmann and Zieme",107.23,1f76529f8574734946361c461b024d99,1325376044,0
2,01/01/19,00:00:51,38859492057661,Lind-Buckridge,220.11,a1a22d70485983eac12b5b88dad1cf95,1325376051,0
3,01/01/19,00:01:16,3534093764340240,"Kutch, Hermiston and Farrell",45.00,6b849c168bdad6f867558c3793159a81,1325376076,0
4,01/01/19,00:03:06,375534208663984,Keeling-Crist,41.96,a41d7549acf90789359a9aa5346dcb46,1325376186,0


In [26]:
# unique_merchants[unique_merchants["merchant"]=="Deckow-O'Conner"]
# unique_purchases[unique_purchases["is_fraud"].isna()==True]

def getPuchaseBatches():
    batches = []
    for i in range(0, len(unique_purchases), 10000):
        batch = unique_purchases.iloc[i:i+10000]
        batches.append(batch)
    return batches

batches = getPuchaseBatches()

In [32]:
batches[2].head()

,trans_date,trans_time,cc_num,merchant,amt,trans_num,unix_time,is_fraud
20000,13/01/19,04:30:51,3560725013359370,"Heathcote, Yost and Kertzmann",189.39,ef92f6da6d6f5ee69b75bee2adb1458d,1326429051,0
20001,13/01/19,04:32:11,6554245334757800,Torp-Labadie,100.66,ef4eb1c898dee144baf3d8d0c0cc22d2,1326429131,0
20002,13/01/19,04:33:13,377234009633447,"Casper, Hand and Zulauf",141.30,acf2069ae66f1d9b28591468b2e0d041,1326429193,0
20003,13/01/19,04:33:21,4635330563105900,Morar Inc,109.48,6d69d460a9bdc4485a45a11970073b67,1326429201,0
20004,13/01/19,04:33:51,4220495028289520000,Jacobi and Sons,3.75,0567a1fd501e0fd619dcca5b376713f0,1326429231,0


## Create connection to Neo4j database



In [8]:
from py2neo import Graph, Node, Relationship

graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))

## Create Function to Load Dataframe records as nodes into GraphDB

In [9]:
def LoadDataframeAsNode(df: pd.DataFrame, label: str, primaryKey: str):
    """
    df : dataframe to be loaded into neo4j as category with records as nodes
    label: name for category in neo4j database
    primaryKey : unique constraint is applied to named property on node 
     to avoid duplicate nodes being created for the same value of named property.
    """

    for _, row in df.iterrows():
        # Create node
        properties = {}
        for column in df.columns:
            properties[column] = row[column]
        node = Node(label, **properties)
        # Merge node in the graph
        graph.merge(node, label, primaryKey)


In [10]:
# load customers into graph database

LoadDataframeAsNode(unique_customers, "Customer", "cc_num")

#load merchants into graph database

LoadDataframeAsNode(unique_merchants, "Merchant", "merchant")

## Establish a relationship between customer node and merchant node based on purchase (transaction)

In [13]:
import concurrent.futures # load module for concurrency

try:
    # get purchases dataframe in batches of size 10 000
    def getPuchaseBatches():
        batches = []
        for i in range(0, len(unique_purchases), 10000):
            batch = unique_purchases.iloc[i:i+10000]
            batches.append(batch)
        return batches


    # create relationship between existing customer and merchant nodes based on purchase batch dataframes
    def createRelation(subsetDF: pd.DataFrame):
        """
        subsetDF : subset of purchase dataframe (batch )
        """
        for _, row in subsetDF.iterrows():
            # Match existing customer and merchant nodes
            customer = graph.nodes.match('Customer', cc_num=row['cc_num']).first()
            merchant = graph.nodes.match('Merchant', merchant=row['merchant']).first()

            # Create the "MADE_PURCHASE_AT" relationship
            relationship = Relationship(customer, "MADE_PURCHASE_AT", 
            merchant, amount=float(row['amt']), date=row["trans_date"],
            time=row["trans_time"], timestamp=row["unix_time"], is_fraud=row["is_fraud"])

            # Create the relationship in the graph
            graph.create(relationship)
        print(f"Done Establishing relationship between Customer and Merchant \
              from transaction_nummber {subsetDF['trans_num'][0]} to \
                transaction_number {subsetDF['trans_num'][-1]}")


    # Create relationships concurrently
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Submit the relationship creation tasks
        results = [
        executor.submit(createRelation, subset)
        for subset in getPuchaseBatches()]
        for f in concurrent.futures.as_completed(results):
            f.result()
except Exception as e:
    print(e)

list index out of range
